# Segmentation en phrases

## Imports

In [1]:
import os
import sys
import nltk
from nltk.tokenize import sent_tokenize

In [2]:
import sys
import nltk
from nltk.tokenize import sent_tokenize

## Fichiers d'inputs et d'outputs

In [3]:
infile = "../tp4/corpus.txt"
outfile = "../tp4/corpustp4.txt"

## Segmentation en phrases du corpus complet et création d'un nouveau fichier

**Important** : pour traiter le corpus complet, indiquez `LIMIT = None`

In [4]:
LIMIT = None

In [5]:
with open(outfile, 'w', encoding="utf-8") as output:
    with open(infile, encoding="utf-8", errors="backslashreplace") as f:
        content = f.readlines()
        content = content[:LIMIT] if LIMIT is not None else content
        n_lines = len(content)
        for i, line in enumerate(content):
            if i % 100 == 0:
                print(f'processing line {i}/{n_lines}')
            sentences = sent_tokenize(line)
            for sent in sentences:
                output.write(sent + "\n")
print("Done")

processing line 0/1
Done


# Word Embeddings : le modèle Word2Vec

## Imports

In [6]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [10]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [11]:
infile = f"../tp4/corpustp4.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [12]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [13]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [14]:
len(bigram_phrases.vocab.keys())

953417

Prenons une clé au hasard :

In [15]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

59_bko


Le dictionnaire indique le score de cette coocurrence :

In [16]:
bigram_phrases.vocab[key_]

1

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [17]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [18]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [19]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [20]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [21]:
print(corpus[:100])

[['deuxieme', 'annee', '.'], ['-', 'n', '.', '130', '.'], ['20_centimes', 'le_numero', 'i', '-', 'liflercrfcdi', '3', 'juin', '1925', ',', 'drapeau', 'telephone', '.'], [':', 'administration', '-', 'publicite', '.', '.', '...', '.', '.', '...', '....,', '16333', '*', 'redaction', '163', '.', '33', '>>', 'redaction', '{', 'apres', '19', 'heures', ')', '296', '.', '89', 'adressa', 'telegraphique', ':', 'drapouge', '-', 'bruxelleo', 'compte', '-', 'cheques_postaux', ';', 'n', '"', '122', '.', '595', 'organe', 'quotidien', 'du_parti', 'communiste', 'beige', '(', 's', 'b', 'i', '57', '-', '5s', 'rue_dea', 'a', '>>', 'eaens', ',', '57', '-', '59', 'bko', '/', 'oellel', '>', '*>,', 'abonnements', ':', 'belgique', ',', '30_septembre', '31', 'decent', '>', 'etranger', '.'], ['1', 'an', '.'], ['i', 't', "'", 'abonne_dans', 'tous_tes', ',', 'bureaux', 'de', 'punis', '.'], ['les', 'dettes_belges', 'envers_les', 'etats_-_unis', 'vers', "l_'_inflation", 'et', 'la_main', '-', 'mise', 'americaine', 's

## Entrainement d'un modèle Word2Vec sur ce corpus

In [22]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=4, # La taille du "contexte", ici 4 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 7 fois dans le corpus
    workers=3, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 3.03 s
Wall time: 7.78 s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [24]:
outfile = f"../tp4/corpustp4.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [26]:
model = Word2Vec.load("../tp4/corpustp4.model")

### Imprimer le vecteur d'un terme

In [27]:
model.wv["banque"]

array([ 0.03437484,  0.2121894 ,  0.49203292,  1.6599562 , -0.0867386 ,
        1.3023146 ,  0.8333397 ,  1.9949716 ,  0.54165703, -0.08588617,
        0.09908707, -3.4434903 ,  0.8731355 ,  1.1881496 ,  2.061644  ,
        1.5250937 , -0.08205003, -1.1289623 , -0.04794616,  2.6105053 ,
        0.7871473 , -0.2297672 ,  2.4509017 , -3.355376  ,  1.1252146 ,
        1.5698082 ,  0.4949429 , -0.59427774,  0.05728798, -0.936747  ,
       -0.8835765 ,  0.18020944], dtype=float32)

In [28]:
model.wv["etat"]

array([-0.12775141, -0.22854167, -0.6612524 ,  0.15537733, -0.1464727 ,
       -0.75726926, -0.841578  ,  0.53345025,  0.1776856 ,  0.3042975 ,
        1.3647552 , -1.6804311 , -0.06034093, -0.571746  , -0.3754234 ,
        0.26141614, -0.07571358,  0.31219697,  0.9429393 ,  0.49520496,
        1.5386255 ,  1.7201954 ,  0.9387401 , -0.5405968 ,  0.597436  ,
       -0.21553768, -0.6214239 , -0.68250006,  1.00557   , -0.5917789 ,
       -0.50789523, -0.2571825 ], dtype=float32)

### Calculer la similarité entre deux termes

In [29]:
model.wv.similarity("cours", "obligations")

0.51527447

In [30]:
model.wv.similarity("obligations", "obligation")

0.6349034

In [31]:
model.wv.similarity("france", "belgique")



0.79148525

### Chercher les mots les plus proches d'un terme donné

In [32]:
model.wv.most_similar("noir", topn=5)

[(':_extra_fancy', 0.9659507870674133),
 ('gaspard', 0.9633066058158875),
 ('gustave', 0.9578896164894104),
 ('toile', 0.9558913111686707),
 ('bleu', 0.9547534584999084)]

In [33]:
model.wv.most_similar("soir", topn=5)

[('pre', 0.949506938457489),
 ('matin', 0.9382753968238831),
 ('siecle', 0.9382133483886719),
 ('capitaine', 0.9332357048988342),
 ('cinema', 0.9332219362258911)]

In [34]:
model.wv.most_similar("actions", topn=5)

[('obligations', 0.8504869937896729),
 ('banques', 0.8275479078292847),
 ('jouissance', 0.8054082989692688),
 ('depots', 0.7919761538505554),
 ('divers', 0.7897666692733765)]

In [35]:
model.wv.most_similar("reine", topn=5)

[('concierge', 0.9580684304237366),
 ('la_lettre', 0.9555862545967102),
 ('dimanche_soir', 0.9539945721626282),
 ('veuve', 0.9538886547088623),
 ('botte', 0.952717661857605)]

In [36]:
model.wv.most_similar("homme", topn=5)

[('prononce', 0.964993417263031),
 ('assure', 0.9625561237335205),
 ('juge', 0.9600217938423157),
 ('tire', 0.9576895236968994),
 ('certainement', 0.9574370980262756)]

In [37]:
model.wv.most_similar("fer", topn=5)

[('des_chemins', 0.830240786075592),
 ('chemin', 0.825129508972168),
 ('actions_des_chemins', 0.7695838212966919),
 ('societe', 0.7685903310775757),
 ('du_chemin', 0.7618299126625061)]

In [38]:
model.wv.most_similar("esclaves", topn=5)

[('prunes', 0.9437491297721863),
 ('academiciens', 0.9427877068519592),
 ('bonifications', 0.9412000179290771),
 ('accueillir', 0.9410020709037781),
 ('opinions', 0.9402633309364319)]

### Faire des recherches complexes à travers l'espace vectoriel

In [39]:
print(model.wv.most_similar(positive=['actions', 'titres'], negative=['belge']))

[('nouvelles', 0.8279739618301392), ('divers', 0.8275811672210693), ('ventes', 0.80405193567276), ('hommes', 0.8008569478988647), ('dix', 0.7914671897888184), ('quatre', 0.7910668253898621), ('livres', 0.788156270980835), ('cinq', 0.7787719964981079), ('fleurs', 0.7755892276763916), ('sieges', 0.7689218521118164)]


In [40]:
print(model.wv.most_similar(positive=['congo', 'affaires'], negative=['belge']))

[('peres', 0.8351684808731079), ('terri', 0.8148234486579895), (';-', 0.8138992786407471), ('enfantine', 0.8127315640449524), ('wallons', 0.8101525902748108), ('combustion', 0.8088886737823486), ('jolis', 0.8087049722671509), ('democrates', 0.8060357570648193), ('ours', 0.8041902184486389), ('depeches', 0.8029871582984924)]


In [41]:
print(model.wv.most_similar(positive=['noir', 'blanc'], negative=['couleur']))

[('lins', 0.8587356209754944), ('73_k', 0.8583097457885742), ('oct', 0.8542829751968384), ('australie', 0.8521870374679565), ('danube', 0.8510916829109192), ('mixed', 0.8495192527770996), ('extra', 0.8468750715255737), ('bahia', 0.8442003130912781), ('bombay', 0.841560959815979), ('canada', 0.8398005366325378)]


In [42]:
print(model.wv.most_similar(positive=['banquier', 'metier'], negative=['communication']))

[('sien', 0.8862278461456299), ('manager', 0.8751124739646912), ('sultan', 0.8750342130661011), ('mur', 0.873248279094696), ('fascisme', 0.8716664910316467), ('requisitoire', 0.8699246644973755), ('prince_leopold', 0.8692768216133118), ('portrait', 0.8679009675979614), ('panorama', 0.8650959134101868), ('prefet', 0.8644734025001526)]
